## NAME : NARIND VERMA
## BATCH : 01 AIML
## SAP ID : 500121898
## ROLL NUMBER: R2142230922

## EXPERIMENT 4: RANDOM FOREST REGRESSOR
 DATA PREPROCESSING

In [13]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

train = pd.read_csv("city_day.csv")
train['AQI'].isnull().sum()

train = train.dropna()
train = train.drop_duplicates()
train.drop(columns=['City'], inplace=True)
train
train.info()
train['Date'] = pd.to_datetime(train['Date'])
train['Year'] = train['Date'].dt.year
train['Month'] = train['Date'].dt.month
train['Day'] = train['Date'].dt.day

# Drop the original 'Date' column
train.drop(columns=['Date'], inplace=True)
train['AQI_Bucket'].unique()
train['AQI_Bucket'] = train['AQI_Bucket'].replace({
    'Moderate': 3,
    'Poor': 2,
    'Very Poor': 1,
    'Satisfactory': 4,
    'Good': 5,
    'Severe': -1,
    ' ': 0
})


<class 'pandas.core.frame.DataFrame'>
Index: 6236 entries, 2123 to 29529
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Date        6236 non-null   object 
 1   PM2.5       6236 non-null   float64
 2   PM10        6236 non-null   float64
 3   NO          6236 non-null   float64
 4   NO2         6236 non-null   float64
 5   NOx         6236 non-null   float64
 6   NH3         6236 non-null   float64
 7   CO          6236 non-null   float64
 8   SO2         6236 non-null   float64
 9   O3          6236 non-null   float64
 10  Benzene     6236 non-null   float64
 11  Toluene     6236 non-null   float64
 12  Xylene      6236 non-null   float64
 13  AQI         6236 non-null   float64
 14  AQI_Bucket  6236 non-null   object 
dtypes: float64(13), object(2)
memory usage: 779.5+ KB


## RANDOM FOREST REGRESSOR


In [14]:
reg = RandomForestRegressor()

In [15]:
params={
        'max_depth':[None,5,6,7,8,10,15,20,30,50], # 10
        'min_samples_leaf':[1,2,5,10,15,20], # 6
        'min_samples_split':[2,5,10,15,20] # 5
        }
rs=RandomizedSearchCV(reg,
                     cv=2,
                     param_distributions=params,
                     scoring='neg_mean_absolute_error',
                     n_iter=50,
                     n_jobs=-1,
                     verbose=2)


In [16]:
X = train[['Year'	,'Month','Day','PM2.5',	'PM10',	'NO',	'NO2',	'NOx'	,'NH3'	,'CO',	'SO2',	'O3',	'Benzene',	'Toluene'	,'Xylene',	'AQI_Bucket']]  # Features
y = train['AQI']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rs.fit(X_train, y_train)

Fitting 2 folds for each of 50 candidates, totalling 100 fits


RandomizedSearchCV(cv=2, estimator=RandomForestRegressor(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'max_depth': [None, 5, 6, 7, 8, 10, 15,
                                                      20, 30, 50],
                                        'min_samples_leaf': [1, 2, 5, 10, 15,
                                                             20],
                                        'min_samples_split': [2, 5, 10, 15,
                                                              20]},
                   scoring='neg_mean_absolute_error', verbose=2)

## EVALUATION


In [17]:
y_pred = rs.best_estimator_.predict(X_test)


In [18]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mse:",mse,"\nR2",r2)

Mse: 225.51754990052592 
R2 0.9728952109107138
